Import necessary packages and functions

In [23]:
import os
import pickle
import random
import time

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

Define parameters

In [29]:
GIT_USERNAME = "dado330"
GIT_MAIL = "messina.davide.statistician@gmail.com"
GIT_REPO = "https://github.com/dado330/AE-WGAN-GP"

train_AE = False
train_WGAN_GP = False
# decoder.trainable = False
train_from_partial = True

batchsize = 20
Z_DIM = 512
random.seed(123)

Git parameter to be run only once

In [25]:
!git clone $GIT_REPO
!git config --global user.email $GIT_MAIL
!git config --global user.name $GIT_USERNAME

fatal: destination path 'AE-WGAN-GP' already exists and is not an empty directory.


Pull changes before running the script

In [34]:
%%bash
cd /content/AE-WGAN-GP
git pull origin main

Already up to date.


From https://github.com/dado330/AE-WGAN-GP
 * branch            main       -> FETCH_HEAD


In [40]:
%%bash
cd /content/AE-WGAN-GP
git add -A
git commit -m "Created folder for generated data populated with still not ok data"
git push -u origin main

On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean
Branch 'main' set up to track remote branch 'main' from 'origin'.


To https://github.com/dado330/AE-WGAN-GP.git
   0834415..274cc33  main -> main


Check if GPU has been loaded correctly

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


Definition of encoder: input is with dimension 18 and has four dense layers of increasing size 64, 128, 256, 512

In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self):
        super(Encoder, self).__init__()
        self.Encoder_DIMS = [64, 128, 256, 512]
        self.dense_layers = [tf.keras.layers.Dense(dim, activation=tf.nn.selu, kernel_regularizer='l1')
                             for dim in self.Encoder_DIMS[:-1]]
        self.output_layer = tf.keras.layers.Dense(self.Encoder_DIMS[-1], activation=tf.nn.sigmoid)

    def call(self, x):
        for s in range(len(self.dense_layers)):
            x = self.dense_layers[s](x)
        x = self.output_layer(x)
        return x

Definition of decoder: input is with dimension 512 and has four dense layers of decreasing size 256, 128, 64, 18

In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self):
        super(Decoder, self).__init__()
        self.Decoder_DIMS = [256, 128, 64, 18]
        self.dense_layers = [tf.keras.layers.Dense(dim, activation=tf.nn.selu, kernel_regularizer='l1')
                             for dim in self.Decoder_DIMS[:-1]]
        self.output_layer = tf.keras.layers.Dense(self.Decoder_DIMS[-1], activation=tf.nn.sigmoid)

    def call(self, x):
        for j in range(len(self.dense_layers)):
            x = self.dense_layers[j](x)
        x = self.output_layer(x)
        return x

Initialize the model for the autoencoder, define the optimizer, create the checkpoint manager and main loss function

In [28]:
ae_optimizer = tf.keras.optimizers.AdamW(learning_rate=0.0001, weight_decay=0.0004,
                                             beta_1=0.7, beta_2=0.99)
encoder = Encoder()
decoder = Decoder()

checkpoint_directory_ae = "/content/AE-WGAN-GP/AE_checkpoints"

checkpoint_ae = tf.train.Checkpoint(encoder=encoder, decoder=decoder)
manager_ae = tf.train.CheckpointManager(checkpoint_ae, directory=checkpoint_directory_ae, max_to_keep=5)

bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)


Define the training step for the autoencoder

In [ ]:
def ae_step(real):
    with (tf.GradientTape() as disc_tape):
        synthetic = decoder(encoder(real))

        main_sparsity_penalty = - tf.math.log(tf.reduce_mean(synthetic + 1e-10)) + tf.math.log(
            tf.reduce_mean(tf.cast(real, tf.float32) + 1e-10))
        sparsity_penalty = - tf.math.log(1 - main_sparsity_penalty)
        autoenc_loss = tf.cast(bce(synthetic, real), tf.float32) + sparsity_penalty
        if np.random.uniform(size=1) > 0.999:
            print("synthetic")
            print(tf.round(synthetic).numpy())
            print("real")
            print(real.numpy())

    gradients_of_autoencoder = disc_tape.gradient(autoenc_loss,
                                                  encoder.trainable_variables + decoder.trainable_variables)
    ae_optimizer.apply_gradients(zip(gradients_of_autoencoder,
                                     encoder.trainable_variables + decoder.trainable_variables))
    return autoenc_loss

Define a custom learning rate scheduler

In [ ]:
def set_learning_rate(epoch):
    if epoch > 500:
        ae_optimizer.learning_rate = 1e-4
    elif epoch > 50:
        ae_optimizer.learning_rate = 3e-4
    elif epoch > 3:
        ae_optimizer.learning_rate = 6e-4

Load the data, divide it in train and test datasets

In [ ]:
data = np.load("/content/AE-WGAN-GP/Input_data/D3_events_ALL_OUTCOMES_ML.npy", allow_pickle=True)
train_data, test_data = train_test_split(data, test_size=0.1)

Train the autoencoder

In [ ]:
if train_AE:
    epochs = 1000
    steps = len(dataset_train)

    for epoch in range(epochs):
        start_time = time.time()
        set_learning_rate(epoch)
        aeloss = 0.0
        for batch_sample in dataset_train:
            aeloss += ae_step(batch_sample)
        duration_epoch_ae = time.time() - start_time
        format_str = 'epoch: %d, aeloss = %.3f (%.2f)'
        print(format_str % (epoch, aeloss / steps / batchsize * 10000, duration_epoch_ae))

    manager_ae.save()
else:
    checkpoint_ae.restore(manager_ae.latest_checkpoint).expect_partial().assert_existing_objects_matched()


Ger the loss of the autoencoder for both train and test data

In [ ]:
aeloss_train = ae_step(train_data)
aeloss_test = ae_step(test_data)
format_str = 'train_aeloss: %.3f, test_aeloss = %.3f'
print(format_str % (aeloss_train / len(train_data) * 10000, aeloss_test / len(test_data) * 10000))

Definition of generator: input is with dimension 512 and has three dense layers of fixed size 512

In [ ]:
class Generator(tf.keras.Model):
    def __init__(self):
        super(Generator, self).__init__()
        self.G_DIMS = [512, 512, 512]
        self.batch_norm_layers = [tf.keras.layers.BatchNormalization(epsilon=1e-5) for _ in self.G_DIMS[:-1]]
        self.dense_layers = [tf.keras.layers.Dense(dim, activation=tf.nn.selu, kernel_regularizer='l1')
                             for dim in self.G_DIMS[1:-1]]
        self.output_layer = tf.keras.layers.Dense(self.G_DIMS[-1], activation=tf.nn.sigmoid)

    def call(self, x, training):
        for k in range(0, len(self.G_DIMS[:-2])):
            x1 = self.dense_layers[k](self.batch_norm_layers[k](x, training=training))
            x += tf.keras.layers.Add()([x1, x])
        x2 = self.output_layer(self.batch_norm_layers[-1](x, training=training))
        x += tf.keras.layers.Add()([x2, x])
        return x

Definition of discriminator: input is with dimension 512 and has five dense layers of decreasing size 256, 128, 64, 32, 1

In [ ]:
class Discriminator(tf.keras.Model):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.D_DIMS = [256, 128, 64, 32]
        self.dense_layers = [tf.keras.layers.Dense(dim, activation=tf.nn.selu) for dim in self.D_DIMS]
        self.output_layer = tf.keras.layers.Dense(1)

    def call(self, x):
        for h in range(len(self.D_DIMS)):
            x = self.dense_layers[h](x)
        x = self.output_layer(x)
        return x

Initialize the model for the encoder, decoder, generator and discriminator

Define the optimizers and create the checkpoint manager

Restore the trained autoencoder

In [ ]:
generator_optimizer = tf.keras.optimizers.AdamW(learning_rate=0.0001, weight_decay=0.0004,
                                                    beta_1=0, beta_2=0.9)
discriminator_optimizer = tf.keras.optimizers.AdamW(learning_rate=0.0001, weight_decay=0.0004,
                                                        beta_1=0, beta_2=0.9)
encoder = Encoder()
decoder = Decoder()
generator = Generator()
discriminator = Discriminator()

checkpoint_directory = "/content/AE-WGAN-GP/WGAN-GP_checkpoints"

checkpoint = tf.train.Checkpoint(encoder=encoder, decoder=decoder, generator=generator, discriminator=discriminator)
manager = tf.train.CheckpointManager(checkpoint, directory=checkpoint_directory, max_to_keep=5)

checkpoint_ae.restore(manager_ae.latest_checkpoint).expect_partial().assert_existing_objects_matched()

Define the gp (gradient penalty)

In [ ]:
def gradient_penalty(batch_size, real_data, fake_data):
    """Calculates the gradient penalty.

    This loss is calculated on an interpolated data row
    and added to the discriminator loss.
    """
    # Get the interpolated data row
    real_data = tf.cast(real_data, tf.float32)
    alpha = tf.random.uniform([batch_size, 1], 0.0, 1.0)
    diff = real_data - fake_data
    interpolated = fake_data + alpha * diff

    with tf.GradientTape() as gp_tape:
        gp_tape.watch(interpolated)
        # 1. Get the discriminator output for this interpolated data row.
        pred = discriminator(interpolated, training=True)

    # 2. Calculate the gradients w.r.t to this interpolated data row.
    grads = gp_tape.gradient(pred, [interpolated])[0]
    # 3. Calculate the norm of the gradients.
    norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=1))
    gp = tf.reduce_mean((norm - 1.0) ** 2)
    return gp

Define the training step for the discriminator/critic

In [31]:
def d_step(real):
    z = tf.random.normal(shape=[batchsize, Z_DIM])
    gp_weight = 10

    with tf.GradientTape() as disc_tape:
        synthetic = decoder(generator(z, training=False))

        real_output = discriminator(real)
        fake_output = discriminator(synthetic)

        disc_cost = tf.reduce_mean(fake_output) - tf.reduce_mean(real_output)

        gp = gradient_penalty(batchsize, real, synthetic)

        disc_loss = disc_cost + gp * gp_weight

    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    return disc_loss

Define the training step for the generator

In [ ]:
def g_step():
    z = tf.random.normal(shape=[batchsize, Z_DIM])
    with tf.GradientTape() as gen_tape:
        synthetic = decoder(generator(z, training=True))

        fake_output = discriminator(synthetic)

        gen_loss = -tf.reduce_mean(fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss,
                                               generator.trainable_variables + decoder.non_trainable_variables)
    generator_optimizer.apply_gradients(
        zip(gradients_of_generator, generator.trainable_variables + decoder.non_trainable_variables))
    return gen_loss

Load the data again in case autoencoder was run in a previous session and to retain only train data (don't need test data anymore)

In [36]:
train_data = np.load("/content/AE-WGAN-GP/Input_data/D3_events_ALL_OUTCOMES_ML.npy", allow_pickle=True)

Train the generator and discriminator/critic

In [ ]:
if train_WGAN_GP:
dloss_list = list()
gloss_list = list()
steps = len(dataset_train)
train_WGAN_GP


if train_from_partial:
    checkpoint.restore(manager.latest_checkpoint).expect_partial().assert_existing_objects_matched()
    with open(os.path.join(checkpoint_directory, "dloss.pkl"), 'rb') as f:
        dloss_list = pickle.load(f)
    with open(os.path.join(checkpoint_directory, "gloss.pkl"), 'rb') as f:
        gloss_list = pickle.load(f)

for epoch in range(10000):
        start_time = time.time()
        dloss = 0.0
        gloss = 0.0
        d_iter = 5
        for batch_sample in dataset_train:
            for _ in range(d_iter):
                dloss += d_step(batch_sample)
            gloss += g_step()

        dloss_list.append(round(dloss.numpy() / (steps * d_iter * batchsize) * 10000, 1))
        gloss_list.append(round(gloss.numpy() / steps / batchsize * 10000, 1))

        duration_epoch = time.time() - start_time
        format_str = 'epoch: %d, dloss = %.1f, gloss = %.1f, total_loss = %.1f (%.2f)'
        print(format_str % (epoch, dloss / (steps * d_iter * batchsize) * 10000, gloss / steps / batchsize * 10000,
         (dloss + gloss) / steps / batchsize * 1000, duration_epoch))

manager.save()
with open(os.path.join(checkpoint_directory, "dloss.pkl"), 'wb') as f:
    pickle.dump(dloss_list, f)
with open(os.path.join(checkpoint_directory, "gloss.pkl"), 'wb') as f:
    pickle.dump(gloss_list, f)

Generate a new set of data

In [ ]:
generator = Generator()
encoder = Encoder()
decoder = Decoder()
discriminator = Discriminator()
checkpoint.restore(manager.latest_checkpoint).expect_partial().assert_existing_objects_matched()


def gen():

    def g_step_generator():
        z = tf.random.normal(shape=[batchsize, Z_DIM])
        synthetic = decoder(generator(z, training=False))

        return synthetic

    synthetic = g_step_generator()
    syn = discriminator(synthetic)
    print(tf.round(synthetic))
    # format_str = 'percentage of generated data classified as true: %d%%'
    # print(format_str % (np.sum(tf.round(syn) == 0) / batchsize * 100))
    # y = tf.gather(tf.round(synthetic), tf.where(tf.squeeze(tf.round(syn) == 0)))
    # print(tf.squeeze(y))

    np.save("/content/AE-WGAN-GP/Generated_data/output_file.npy", train_data)

In [ ]:
encoder = Encoder()
encoder.summary()

Plot the discriminator loss

In [ ]:
checkpoint_directory = "/content/gdrive/My Drive/Colab_Notebooks/training_checkpoints_medgan"
with open(os.path.join(checkpoint_directory, "dloss.pkl"), 'rb') as f:
  dloss_list = pickle.load(f)
fig, ax = plt.subplots()
ax.plot(dloss_list)
plt.title('Discriminator/Critic Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend('d-loss', loc='lower right')
plt.show()
fig, ax = plt.subplots()
ax.plot(dloss_list)
ax.set_ylim(-200, 200)
plt.title('Discriminator/Critic Loss (Zoom in)')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend('d-loss', loc='lower right')
plt.show()

Plot the generator loss

In [ ]:
with open(os.path.join(checkpoint_directory, "gloss.pkl"), 'rb') as f:
  gloss_list = pickle.load(f)
plt.plot(gloss_list)
plt.title('Generator Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend('g-loss', loc='lower right')
plt.show()